# Lab3.4 Sentiment Analysis with machine-learning

Copyright: Vrije Universiteit Amsterdam, Faculty of Humanities, CLTL

The focus of this notebook is on performing sentiment analysis using the scikit-learn package. Material from [this notebook](http://www.pitt.edu/~naraehan/presentation/Movie+Reviews+sentiment+analysis+with+Scikit-Learn.html) was re-used and adapted.

**At the end of this notebook, you will be able to**:
* load the training data, i.e., the movie reviews
* inspect the training data, i.e., the movie reviews
* extracting features from the training data
* training and evaluating the *NaiveBayesClassifier*
* apply the classifier to fake movie reviews


For training a machine learning system we need a number of packages, the most important ones are *sklearn* and *numpy* to manipulate data and call machine learning functions. Since we are dealing with texts, we also need some specific packages from *sklearn* to operate on texts to get words as features.

In [1]:
import sklearn
import numpy
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import load_files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

## Loading a dataset
We are first going to load and inspect the **airlinetweets** dataset (which is included in the data folder that you downloaded from HLT course Github). This data set originates from:

https://www.kaggle.com/crowdflower/twitter-airline-sentiment

Kaggle is a platform where you can find many more data sets.

We are going to use the method **load_files** as part of sklearn.
Let's first inspect what the help message of the function **load_files** states.

In [2]:
help(load_files)

Help on function load_files in module sklearn.datasets._base:

load_files(container_path, *, description=None, categories=None, load_content=True, shuffle=True, encoding=None, decode_error='strict', random_state=0)
    Load text files with categories as subfolder names.
    
    Individual samples are assumed to be files stored a two levels folder
    structure such as the following:
    
        container_folder/
            category_1_folder/
                file_1.txt
                file_2.txt
                ...
                file_42.txt
            category_2_folder/
                file_43.txt
                file_44.txt
                ...
    
    The folder names are used as supervised signal label names. The individual
    file names are not important.
    
    This function does not try to extract features into a numpy array or scipy
    sparse matrix. In addition, if load_content is false it does not try to
    load the files in memory.
    
    To use text files in a sc

Ok, so the function requires the following structure of the data on your disk in order for it to work:

* container_folder/
    * category_1_folder/ (e.g., 'positive')
        * file_1.txt
        * file_2.txt
        * ...
        file_42.txt
    * category_2_folder/ (e.g., 'negative')
        * file_43.txt
        * file_44.txt
        * ...
        
The names of the subfolders are treated as the labels for the data and you are supposed to divided the text files over the subfolders accordingly.

Let's check whether our **airlinetweets** corpus has this structure. In the data folder, you will see a folder "airlinetweets" with 3 subfolders: "negative", "neutral", and "positive". These subfolder names will represent our sentiment labels. In each folder, you find a long list of text files, each containing a tweet. This is our training and test data per category.

We will use the **pathlib** package to get the path of this notebook and extend it with "data/airlinetweets", assuming that you stored the data in this location. If this is not the case, you need to adapt the path accordingly.

In [3]:
import pathlib
cwd = pathlib.Path.cwd()
airline_tweets_folder = cwd.joinpath('data/airlinetweets')
print('path:', airline_tweets_folder)
print('this will print True if the folder exists:', 
      airline_tweets_folder.exists())

path: /Users/piek/Desktop/t-ONDERWIJS/2021-2022/t-MA-HLT-introduction-2021/ma-hlt-labs/lab3.machine_learning/data/airlinetweets
this will print True if the folder exists: True


In [4]:
str(airline_tweets_folder)

'/Users/piek/Desktop/t-ONDERWIJS/2021-2022/t-MA-HLT-introduction-2021/ma-hlt-labs/lab3.machine_learning/data/airlinetweets'

Inspect on your local machine whether the data has the required structure. Take your time to open files and inspect them. It is always good to know what forms your data.

....

....

Hopefully, it is! Let's now load it using the **load_files** function.

In [5]:
# loading all files as training data.
airline_tweets_train = load_files(str(airline_tweets_folder))

## Inspecting dataset

The load_files function will read each file separately and store it as a data item or instance. The data is internally stored as a specfic type of object called *Bunch*, which consists of a description 'DESCR', the names of the loaded files, the target data and the category labels (target_names).

In [6]:
print(type(airline_tweets_train))
print(dir(airline_tweets_train))
print(type(airline_tweets_train.target))
print('Target names', airline_tweets_train.target_names)

<class 'sklearn.utils.Bunch'>
['DESCR', 'data', 'filenames', 'target', 'target_names']
<class 'numpy.ndarray'>
Target names ['negative', 'neutral', 'positive']


How many files do we have? Well, that would be the same as the number of data items.

In [7]:
print('Number of data items created from the laoded files', len(airline_tweets_train.data))

Number of data items created from the laoded files 4755


If you do not like with the data labels, you could change the names of the subdirectories. If you do not agree with the distinctions, you could add other folders with other category names and move files to these folders.

Note that *target* is a **numpy.ndarray** that represents the index to the category values for all the data that we have loaded, in this case all the tweet files: 4,755 in total. The category values are represented in a list of target_names so the value '0' represents 'negative', '1' represents 'neutral' and '2' represents 'positive'. Let's have a look. Because it is a *numpy* object, we need to convert it to a list to print it.

In [8]:
print(len(airline_tweets_train.target.tolist()))
print(airline_tweets_train.target.tolist())

4755
[1, 1, 2, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 2, 1, 0, 0, 2, 0, 2, 2, 2, 1, 2, 1, 0, 2, 2, 0, 0, 1, 0, 2, 0, 0, 1, 1, 1, 1, 1, 2, 0, 0, 0, 1, 0, 0, 1, 2, 0, 2, 0, 2, 0, 1, 0, 2, 2, 2, 0, 1, 2, 2, 2, 2, 0, 1, 1, 2, 1, 0, 0, 1, 1, 1, 0, 0, 2, 0, 1, 0, 1, 2, 1, 0, 2, 2, 0, 2, 0, 0, 2, 2, 0, 2, 2, 0, 1, 0, 1, 0, 0, 0, 0, 1, 2, 1, 2, 2, 2, 1, 0, 2, 2, 0, 0, 2, 0, 0, 1, 1, 1, 2, 1, 0, 0, 2, 0, 1, 1, 0, 1, 2, 1, 0, 2, 1, 0, 2, 0, 1, 0, 2, 1, 2, 0, 1, 1, 2, 0, 2, 2, 0, 2, 2, 0, 1, 1, 0, 2, 1, 1, 1, 2, 1, 2, 1, 0, 0, 1, 2, 1, 2, 2, 0, 2, 1, 0, 2, 0, 2, 0, 1, 2, 2, 0, 2, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 2, 2, 1, 1, 1, 0, 1, 1, 2, 1, 2, 2, 1, 0, 2, 0, 2, 0, 2, 0, 0, 0, 1, 2, 0, 0, 1, 1, 1, 1, 2, 0, 1, 2, 2, 1, 0, 1, 0, 1, 1, 2, 0, 2, 0, 2, 1, 0, 0, 2, 0, 0, 0, 2, 1, 0, 2, 2, 0, 1, 2, 0, 0, 2, 1, 2, 1, 0, 1, 2, 2, 2, 2, 2, 2, 1, 1, 0, 0, 0, 2, 2, 2, 0, 2, 1, 2, 1, 1, 2, 0, 1, 1, 2, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 2, 1, 1, 0, 2, 1, 2, 0, 2, 0, 0, 0, 0, 2, 2, 1, 0, 2, 0, 0, 1, 1, 2, 0, 2, 0

In this case, the numbers are not the vector representations of the words!!

We see a sequence of 0, 1, 2 values which represents the sequence of categories for the sequence of loaded texts. The first two are neutral (1) and the third is positive (2, the fifth negative (0).

Let's is look at the actual text instances. What is the first text, which is supposed to be neutral:

In [9]:
# Let's inspect the first data element
airline_tweets_train.data[0]

b'@AmericanAir Why is your cover photo of TWA? Just wondering.'

In [10]:
# which comes from the first file is in "neutral" folder
airline_tweets_train.filenames[0]

'/Users/piek/Desktop/t-ONDERWIJS/2021-2022/t-MA-HLT-introduction-2021/ma-hlt-labs/lab3.machine_learning/data/airlinetweets/neutral/AL_570069345818161152.txt'

In [11]:
# first file is a neutral review and is mapped to index 1 in target_names
airline_tweets_train.target[0]

1

We can find out what the index means by inserting it into **target_names**

In [12]:
airline_tweets_train.target_names[1]

'neutral'

How many do we have for each category? We can use the **Counter** package in Python to get statistics from the target structure by counting each category value and displaying the category using the index:

In [13]:
from collections import Counter

freqs = Counter(airline_tweets_train.target)
print(type(freqs))
for category, frequency in freqs.items():
    print(category, frequency)
    print(airline_tweets_train.target_names[category], frequency)

<class 'collections.Counter'>
1 1515
neutral 1515
2 1490
positive 1490
0 1750
negative 1750


We can see that the data are equally distributed over the three categories, which is good!

## Loading another data set

Through the **load_files** function, we can easily load any text collection per category by putting them into separate subfolders.  Let's try another one. Remember from the first Lab that some of the NLTK data is structured precisely in the same way as the airlinetweets data, e.g. nltk_data/corpora/movie_reviews.

Likewise, we can load this data in the same way using the load_files function. First adapt the path below to point to the location on your local laptop.

In [14]:
movie_reviews_folder = cwd.joinpath('/Users/piek/nltk_data/corpora/movie_reviews')
print('path:', movie_reviews_folder)
print('this will print True if the folder exists:', 
      movie_reviews_folder.exists())

path: /Users/piek/nltk_data/corpora/movie_reviews
this will print True if the folder exists: True


In [15]:
movie_reviews_train = load_files(str(movie_reviews_folder))

In [16]:
print(type(movie_reviews_train))
print(dir(movie_reviews_train))
print('Number of data items', len(movie_reviews_train.data))
print('Target names', movie_reviews_train.target_names)
print('First data record', movie_reviews_train.data[0])

<class 'sklearn.utils.Bunch'>
['DESCR', 'data', 'filenames', 'target', 'target_names']
Number of data items 2000
Target names ['neg', 'pos']
First data record b"arnold schwarzenegger has been an icon for action enthusiasts , since the late 80's , but lately his films have been very sloppy and the one-liners are getting worse . \nit's hard seeing arnold as mr . freeze in batman and robin , especially when he says tons of ice jokes , but hey he got 15 million , what's it matter to him ? \nonce again arnold has signed to do another expensive blockbuster , that can't compare with the likes of the terminator series , true lies and even eraser . \nin this so called dark thriller , the devil ( gabriel byrne ) has come upon earth , to impregnate a woman ( robin tunney ) which happens every 1000 years , and basically destroy the world , but apparently god has chosen one man , and that one man is jericho cane ( arnold himself ) . \nwith the help of a trusty sidekick ( kevin pollack ) , they will

So basically, we get a similar data structure with text data and the labels "neg" and "pos".

## Extracting features from training data


We now have loaded two data sets as instances of the object <class 'sklearn.utils.Bunch'> in our notebook. Next, we are going to use the CountVectorizer function to process the text data and to create a **Bag-of-Words** presentation as we have done before for the toy data. We define a separate instance of CountVectorizer for each data set. 

In [17]:
# initialize airline object, and then turn airline tweets train data into a vector 

airline_vec = CountVectorizer(min_df=10, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english')) # stopwords are removed

In order to represent each document in terms of this vector, we use the *fit_transform* function to generate a matrix of documents (the rows) and the vectors with the scores for each words that occurs in each document. The *fit_transform* function creates the model from the vocabulary and applies some transformation to it. You can also use the *fit* function.

Note: you might get a warning when you run the following cell. You do NOT have to resolve the warning.

In [18]:
airline_counts = airline_vec.fit_transform(airline_tweets_train.data)

/Users/piek/opt/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


We have now created a vector representation *airline_vec* of the complete vocabulary of the full data set. Every position in this vector represents a unique word token.

In [19]:
#Total number of word features or the length of the total vector
print('Total number of words in our data:', len(airline_vec.vocabulary_))

Total number of words in our data: 653


In [20]:
# First 50 feature names
print(list(airline_vec.get_feature_names())[:50])

['!', '#', '$', '&', "'", "''", "'d", "'ll", "'m", "'re", "'s", "'ve", '(', ')', '-', '--', '.', '..', '...', '....', '1', '10', '100', '15', '1k', '1st', '2', '20', '200', '2015', '24', '25', '2nd', '3', '30', '4', '40', '45', '5', '50', '6', '7', '8', ':', ';', '?', '@', '``', 'aa', 'able']


/Users/piek/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [21]:
# 'plane' is found in the corpus, mapped to index 1948
airline_vec.vocabulary_.get('plane')

443

We can now inspect the dimensions of our feature array by getting the spape: the rows (documents) and columns (the word vector length).

In [22]:
# large dimensions! 4,755 documents, 2902 unique terms. 
airline_counts.shape

(4755, 653)

It is important to inspect the dimensions of the data matrix. We see that we have 4,755 text instances with each a vector array of 2,902 positions. Each position represents a word from the training data for which we can set a value that indicates if or how often it occurs in a text instance.

We can convert the matrix to an array and get the first element and look at the vector values for slots 100 till 200:

In [23]:
print(airline_counts.toarray()[0][10:600])

[0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 

We can see that most values are zero's and just a few have the value 1. This is what we call a *sparse vector*.

As we have seen in the previous Lab, we can also transform the counts into information value scores using the *TfidfTransformer* function.

In [24]:
# Convert raw frequency counts into TF-IDF values
tfidf_transformer = TfidfTransformer()
airline_tfidf = tfidf_transformer.fit_transform(airline_counts)

Obviously the shape remains the same but the values are now weighted scores between zero and one.

In [25]:
# Same dimensions, now with tf-idf values instead of raw frequency counts
datarow = 30
print(airline_tfidf.shape)
# we print values 100:250 for datarow 
print(airline_tfidf.toarray()[datarow][10:250])

(4755, 653)
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.33609914
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.26745264
 0.         0.28262499 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.0520561  0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.   

## Training and testing a Naive Bayes classifier for airline tweets

We can now use the above data representation as training data to build a classifier. the Sklearn package already associated each row (a document) in our data represenation with a label by taking the name of the data subfolder.

In [26]:
print("Label for data at row:", airline_tweets_train.target[datarow])
print("Name for the label as number:", airline_tweets_train.target_names[airline_tweets_train.target[datarow]])

Label for data at row: 0
Name for the label as number: negative


We are going to use a simple Naive Bayes classifier to train a model. A Naive Bayes (NB) classifier is a so-called generative classifier that learns the probablity of a word given a category. By assessing the words a text it can aggregate how likely a text is for a category. You will learn more about this type of classifier in the machine learning code. For now it is enough to know that NB work reasonably for smaller data sets and with little training.  

Because we have multiple labels (negative, positive, neutral) and therefore are dealing with 3 classes, we need a *multinomial classifier*.

In [27]:
# Now ready to build a classifier. 
# We will use Multinominal Naive Bayes as our model
from sklearn.naive_bayes import MultinomialNB

It is easy for machine learning package to read the above vector representations and associated these with any type of label. However, we also want to test the data. For that purpose, we need to exclude part of the data from a training set.

To train the classifier, we will first split the data into train and test. Sklearn has a built in function for this that we need to import.

In [28]:
# Split data into training and test sets
# from sklearn.cross_validation import train_test_split  # deprecated in 0.18
from sklearn.model_selection import train_test_split

We choose 80% training and 20% test. The function returns 4 different data structures that we need to catch: the training texts, the test texts, the training labels and the test labels. In Python, we can assign these to four different variables in one command call. The *train_test_split* function requires as input the model, the data and the train-test proportional split.

In [29]:
docs_train, docs_test, human_labels_train, human_labels_test = train_test_split(
    airline_tfidf, # the tf-idf model
    airline_tweets_train.target, # the tweets and their labels
    test_size = 0.20 # we use 80% for training and 20% for development
    ) 

We now find our training texts in *docs_train* and our test texts in *docs_test*. The corresponding sequence of labels for training are in *human_labels_train*, and those for testing are in *human_labels_test*.

In [30]:
print("Training texts:", len(docs_train.toarray()))
print("Training labels:", len(human_labels_train))
print("Test texts:", len(docs_test.toarray()))
print("Test labels:", len(human_labels_test))

Training texts: 3804
Training labels: 3804
Test texts: 951
Test labels: 951


We see that we have an equal numer of texts and labels for train and test. The values should be in the right order.

One training instance looks like this:

In [31]:
docs_train[55].toarray()

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.68023261, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.13505632, 0.68046026, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.  

it's label is:

In [32]:
human_labels_train[55]

2

which we know is then:

In [33]:
airline_tweets_train.target_names[human_labels_train[55]]

'positive'

The *fit* function of sklearn takes as input the training data and the labels.

In [34]:
# Train a Multimoda Naive Bayes classifier
airline_tweets_clf = MultinomialNB().fit(docs_train, human_labels_train)

After training the classifier, we can apply it to test data that is represented in the same way.

In [35]:
# Predicting the Test set results, find macro recall
system_labels_pred = airline_tweets_clf.predict(docs_test)

We can now obtain each test tweet

In [37]:
test_item = 20
#### we need to get the text from the original text data before we did the split.
#### Remember that the nTH item in an array has an index of n-1 because we start the index with zero.
#### Likewise, we take the length of the training data minus 1, to find the start of the test data.
data_index = len(docs_train.toarray())-1+test_item
print('Tweet number:', data_index)
print('The test tweet:', airline_tweets_train.data[data_index])
print('one test tweet representation:', docs_test[test_item-1].toarray().tolist())
print('gold label:', human_labels_test[test_item-1])
print('classifier predicted:', system_labels_pred[test_item-1])

Tweet number: 3823
The test tweet: b'"Bruh...real tweet from @JetBlue ""Our fleet\'s on fleek. http://t.co/dqny4aKTg9"""'
one test tweet representation: [[0.20108888036631759, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1551587774718064, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16845021625429452, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

### Measuring the quality of the system output against the human labels

Sklearn provides functions to obtain the recall, precision and f-measure for the test set results. We can simply pass the human labels and the system labels as input and set a parameter how to aggregate the overall score. Separate functions are provide for the recall, precision and f-measure.

In [61]:
sklearn.metrics.recall_score(y_true=human_labels_test,
                             y_pred=system_labels_pred,
                             average='micro')

0.8233438485804416

The classification_report function gives the complete result:

In [62]:
from sklearn.metrics import classification_report

report = classification_report(human_labels_test,system_labels_pred,digits = 3)
print(report)

              precision    recall  f1-score   support

           0      0.810     0.890     0.848       364
           1      0.865     0.693     0.770       313
           2      0.807     0.883     0.843       274

    accuracy                          0.823       951
   macro avg      0.827     0.822     0.820       951
weighted avg      0.827     0.823     0.821       951



Whenever you generate a report such as this, don't just stare at the overall performance such as *accuracy* but consider also the results per category and what the relation is between recall and precision. Ideally, recall and precision are balanced but if they deviate it is worth thinking about the causes. Some good causes are:

<ol>
<li>Not enough training data to capture all variation in the test data
<li>Biased training data that makes one category stronger than the other
<li>Too much ambiguity so that words as features have different interpretions in different contexts
<li>Over-fitting to the training data so that it learns features that do not generalise to the test data
<li>Many others....
</ol>

During this master programme, you will work with many different data sets and analyse the performance of classifiers. Analysing different data, experimenting with text representations, settings and machine learning algorithm is the best way to learn about possible causes and how to mitigate them.

In the case of a Naive Base classifier, we can easily inspect the least and most important features per category.

In [63]:
def important_features_per_class(vectorizer,classifier,n=10): #n is the number of top features
    class_labels = classifier.classes_
    feature_names =vectorizer.get_feature_names()
    topn_class1 = sorted(zip(classifier.feature_count_[0], feature_names),reverse=True)[:n]
    topn_class2 = sorted(zip(classifier.feature_count_[1], feature_names),reverse=True)[:n]
    topn_class3 = sorted(zip(classifier.feature_count_[2], feature_names),reverse=True)[:n]
    print("Important words in negative documents")
    for coef, feat in topn_class1:
        print(class_labels[0], coef, feat)
    print("-----------------------------------------")
    print("Important words in neutral documents")
    for coef, feat in topn_class2:
        print(class_labels[1], coef, feat) 
    print("-----------------------------------------")
    print("Important words in positive documents")
    for coef, feat in topn_class3:
        print(class_labels[2], coef, feat) 

# example of how to call from notebook:
important_features_per_class(airline_vec, airline_tweets_clf)

Important words in negative documents
0 156.93427869254464 united
0 112.1616409687085 .
0 96.91903066761333 @
0 93.27171751986671 ``
0 57.0315819812597 flight
0 50.203424483318386 ?
0 44.273490871415554 !
0 43.86464647317689 #
0 38.72394459013907 n't
0 27.717300421533164 ''
-----------------------------------------
Important words in neutral documents
1 105.24952188894775 @
1 84.30196140256247 ?
1 62.434258876875454 jetblue
1 58.56264189918974 southwestair
1 57.17709947137887 .
1 57.03907744236674 ``
1 54.57084502075883 americanair
1 52.52222634150541 :
1 41.9752297371489 usairways
1 37.63956254791795 flight
-----------------------------------------
Important words in positive documents
2 171.6494499723948 !
2 106.11764629892475 @
2 88.15409338193925 .
2 83.85020396837129 thanks
2 79.31943694585807 thank
2 68.36570020375369 jetblue
2 67.83584928111172 southwestair
2 65.79361585443769 ``
2 50.10652350217669 #
2 49.18628819696222 americanair


What do you think about these most important words? According to the algorithm these words are most strongly associated with the categories. Is this intuitive to you?

By inspecting the performance and the feature analysis, you will start gaining insight in the data and the system. Hopefully this inspires you how to improve the system performance.

## Applying the airline tweets classifier on your own data
Now we can apply our classifier to new data. Let's type a few potential tweets.

In [64]:
# very short and fake movie reviews
my_reviews = ['This movie was excellent', 
               'Absolute joy ride', 
               'Steven Seagal was terrible', 
               'Steven Seagal shined through.', 
               'This was certainly a movie', 
               'Two thumbs up', 
               'I fell asleep halfway through', 
               "We can't wait for the sequel!!", 
               'I cannot recommend this highly enough', 
               'instant classic.', 
               'Steven Seagal was amazing.']
len(my_reviews)

11

To be able to apply our model to these text, we need to represent the text using the same vector dimensions (columns!) as we used for training the model.

The sklearn transformer function does this work for you using the **transform** function. This function takes the vectors for representing the training data. Note that these vectors have a position for every word in the training data. So we cannot represent words that do not occur outside the training data, so-called Out-of-Vocabulary words. Technically, we do not need to worry because the **transform** function will take care of this but you can imagine that texts that are very different from the training data will be represented poorly by the model.

In [65]:
# We re-use airline_vec to transform it in the same way as the training data
my_review_counts = airline_vec.transform(my_reviews)
my_review_counts.shape

(11, 2902)

We can see that the shape of our matrix represents 11 rows using the vectors with 2,907 dimensions that we created from the training data before. So words in our movie reviews that are NOT in the training data, are not represented as there are no slots in the vectors from the training data.

We can see this clearly, if we would use the **fit_transform** function instead of **transform** for our tweets, which will create a whole new model from our 11 tweets.

In [66]:
bad_my_review_counts = airline_vec.fit_transform(my_reviews)
bad_my_review_counts.shape

(11, 4)

We see that we only have 4 dimensions left because the CountVectorizer created complete new vector representations using our settings: min_df=2,tokenizer=nltk.word_tokenize, stop_words=stopwords.words('english'). The vector only represents the words from my_reviews and this representation is incompatible with our trained model. So we cannot use this representation!!

This is a mistake you are likely to make. Remember:

1. build a vectore representation from the features from the training data: use **fit** or **fit_transform** with your defined CountVectorizer instance
2. train a classifier from these representation
3. always, always use the **transform** function from your defined CountVectorizer instance (which has been derived using the training data) to represent any other texts thatr you want to classify

It is good practice to always check the **shape** function if the texts are represented using the same number of dimensions.

Assuming that our own review texts are now propery represented in my_review_counts*my_review_counts*, we can proceed with calculating **TF\*IDF** values and use our trained classifier to make predictions on these representations.

In [67]:
# we compute tf idf values
my_reviews_tfidf = tfidf_transformer.transform(my_review_counts)

In [68]:
my_reviews_tfidf.shape

(11, 2902)

These dimensions are correct so let's get the predictions:

In [69]:
# have classifier make a prediction
pred_on_my_reviews = airline_tweets_clf.predict(my_reviews_tfidf)

In [70]:
# print out results ()
for review, predicted_label in zip(my_reviews, pred_on_my_reviews):
    
    print('%s => %s' % (review, 
                        airline_tweets_train.target_names[predicted_label]))

This movie was excellent => positive
Absolute joy ride => positive
Steven Seagal was terrible => negative
Steven Seagal shined through. => negative
This was certainly a movie => negative
Two thumbs up => negative
I fell asleep halfway through => neutral
We can't wait for the sequel!! => negative
I cannot recommend this highly enough => negative
instant classic. => negative
Steven Seagal was amazing. => positive


Take a minute to see if these reviews are correctly classified. What would be needed to evaluate the performance?

## Training another classifier with movie reviews

Remember that we loaded the NLTK movie review data set using the sklearn function *load_files* in the same way as we have done for the airlinetweets. This means we can build another classifier from this data as well and apply it to the same set of *my_reviews* and compare the two systems.

We proceed in three simple steps:

<ol>
    <li>We create a CountVectorizer to vectorize the training texts based on the total vocabulary using the *fit_transform* function
    <li>We transfer the values using the tfidf_transformer into information values
    <li>We create a *MultinomialNB* classifier from the vectorised documents and their labels
</ol>

Once we trained the classifier, we can apply it to the above examples by transforming the my_review texts to this reptentation to make them compatible to our new model.

In [71]:
movie_vec = CountVectorizer(min_df=2, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english')) # stopwords are removed

Using this instance of CountVectorizer, we can now derive a new (and very different) vector representation from the movie_review data we loaded before. We use here the **fit_transform** function.

In [72]:
movie_counts = movie_vec.fit_transform(movie_reviews_train.data)

/Users/piek/opt/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


We can use **shape** to see that our matrix has 2000 rows for the data, which are now represented through 25,138 dimensions. So this is a very different vector representation than derived from the airline tweets.

In [73]:
movie_counts.shape

(2000, 25138)

We convert the count values to **TF\*IDF** values as we did before.

In [74]:
movie_tfidf = tfidf_transformer.fit_transform(movie_counts)

The sklearn load_files functions not only return the data for training but also the classification labels that go with it based in the folder names that contain the text files. The *target* attribute contains the lists of values as integer indexes corresponding to each document representation in the data and *target_names* contain the meaning of the labels.

In [75]:
print(movie_reviews_train.target_names)
# print the labels for the first ten documents
print(movie_reviews_train.target[:10])

['neg', 'pos']
[0 1 1 0 1 1 1 1 1 0]


In [76]:
movie_review_clf = MultinomialNB().fit(movie_tfidf, movie_reviews_train.target)

In [77]:
## Applying the movier review classifier to your own data

To apply our new classifier to the same my_reviews data, we first need to represent these texts according to the vectors of the movie_review data. So, we call again the **transform* function 

In [78]:
# We re-use airline_vec to transform it in the same way as the training data
my_review_movie_review_counts = movie_vec.transform(my_reviews)
my_review_movie_review_counts.shape

(11, 25138)

In [79]:
reviews_new_tfidf = tfidf_transformer.transform(my_review_movie_review_counts)

In [80]:
movie_review_clf_pred = movie_review_clf.predict(reviews_new_tfidf)

In [81]:
# print out results ()
for review, predicted_label in zip(my_reviews, movie_review_clf_pred):
    
    print('%s => %s' % (review, 
                        movie_reviews_train.target_names[predicted_label]))

This movie was excellent => pos
Absolute joy ride => pos
Steven Seagal was terrible => neg
Steven Seagal shined through. => neg
This was certainly a movie => neg
Two thumbs up => neg
I fell asleep halfway through => neg
We can't wait for the sequel!! => neg
I cannot recommend this highly enough => pos
instant classic. => pos
Steven Seagal was amazing. => neg


Our moview review classifier only uses two categories instead of the airlinetweets classifier. Is it doing any better?

## End of this notebook